## Import libraries

In [ ]:
#from keras.utils import to_categorical
from keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation, concatenate
#from keras.layers.advanced_activations import LeakyReLU
from keras.layers import LeakyReLU
#from keras.models import Model
from keras.callbacks import ModelCheckpoint


import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import shutil 
import matplotlib.pyplot as plt
#import tensorflow_datasets as tfds
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline
from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import EfficientNetB0
#from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.optimizers import Adam
import math

## EfficentCNN Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dense, Dropout, Flatten, concatenate
from tensorflow.keras.models import Model

class CNN(tf.keras.Model):
    def __init__(self):
        super(CNN, self).__init__()
        self.base = tf.keras.applications.EfficientNetB0(input_shape=(75, 75, 3),
                                                        include_top=False,
                                                        weights='imagenet')
        self.base.trainable = True
        self.conv = Conv2D(3, 3, padding='same')
        self.conv1 = Conv2D(16, (3, 3), padding='same', activation='relu')
        self.batchnorm1 = BatchNormalization()
        self.pool1a = MaxPooling2D((2, 2), strides=(2, 2))
        
        self.conv2a = Conv2D(32, (3, 3), padding='same', activation='relu')
        self.conv2b = Conv2D(32, (3, 3), padding='same', activation='relu')
        self.batchnorm2 = BatchNormalization()
        self.pool2a = MaxPooling2D((2, 2), strides=(2, 2))
        self.pool2b = MaxPooling2D((2, 2), strides=(2, 2))
        self.pool2c = MaxPooling2D((2, 2), strides=(2, 2))
        
        self.conv3a = Conv2D(64, (3, 3), padding='same', activation='relu')
        self.conv3b = Conv2D(64, (3, 3), padding='same', activation='relu')
        self.drop3 = Dropout(0.3)
        self.batchnorm3 = BatchNormalization()
        self.pool3a = MaxPooling2D((2, 2), strides=(2, 2))
        
        self.conv4a = Conv2D(128, (3, 3), padding='same', activation='relu')
        self.conv4b = Conv2D(128, (3, 3), padding='same', activation='relu')
        self.conv4c = Conv2D(128, (3, 3), padding='same', activation='relu')
        self.drop4 = Dropout(0.3)
        self.batchnorm4 = BatchNormalization()
        self.pool4a = MaxPooling2D((2, 2), strides=(2, 2))
        #self.skip3 = concatenate([self.pool2c, self.pool4a])
        
        self.conv5a = Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv5b = Conv2D(256, (3, 3), padding='same', activation='relu')
        self.conv5c = Conv2D(256, (3, 3), padding='same', activation='relu')
        self.batchnorm5 = BatchNormalization()
        self.pool5a = MaxPooling2D((2, 2), strides=(2, 2))
        
        self.flatten = Flatten()
        self.dense1 = Dense(256, activation='relu')
        self.output_layer = Dense(10)

    def call(self, x, training=False):
        #x = self.conv(x)
        #x = self.base(x)
        x = self.conv1(x)
        x = self.batchnorm1(x, training=training)
        x = self.pool1a(x)
        
        x = self.conv2a(x)
        x = self.conv2b(x)
        x = self.batchnorm2(x, training=training)
        x = self.pool2a(x)
        x = self.pool2b(x)
        x = self.pool2c(x)
        
        x = self.conv3a(x)
        x = self.conv3b(x)
        x = self.drop3(x, training=training)
        x = self.batchnorm3(x, training=training)
        x = self.pool3a(x)
        
        x = self.conv4a(x)
        x = self.conv4b(x)
        x = self.conv4c(x)
        x = self.drop4(x, training=training)
        x = self.batchnorm4(x, training=training)
        x = self.pool4a(x)
        #x = self.skip3()
        
        x = self.conv5a(x)
        x = self.conv5b(x)
        x = self.conv5c(x)
        x = self.batchnorm5(x, training=training)
        #x = self.pool5a(x)
        
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.output_layer(x)
        return x

    def build_graph(self):
        x = tf.keras.Input(shape=(75, 75, 4))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))

model = CNN()
model.build(input_shape=(None, 75, 75, 4))

model.summary()
